In [1]:
#!pip install gym
#!pip install pygame

In [2]:
import gym
import numpy as np

In [3]:
env = gym.make('CartPole-v1', render_mode='human')

/Users/juanpaps03/Library/Caches/pypoetry/virtualenvs/jupyter-env-X9eyKUcs-py3.10/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/juanpaps03/Library/Caches/pypoetry/virtualenvs/jupyter-env-X9eyKUcs-py3.10/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    if explore:
        action = env.action_space.sample()
        print('explore')
    else:
        action = np.argmax(Q[state])
        print('exploit')
    return action

In [ ]:
def q_learning(state, Q, epsilon=0.1):
    '''
    Inicializar Q(s, a) arbitrariamente ∀s ∈ S, a ∈ A(s)
    Repetir:
        Inicializar s
        done ← False
        Repetir hasta done:
            Con probabilidad ε hacer: (* estrategia ε-greedy *)
                explore: a ← sample(A(s))
                exploit: a ← arg m ́ax Q(s, ·)
            s′, r , done ← step(a)
            Q(s, a) ← Q(s, a) + α(r + γ max Q(s′, ·) − Q(s, a))
            s ← s′
    '''
    
    done = False
    while not done:
        action = epsilon_greedy_policy(state, Q, epsilon)
        obs, reward, done, info = env.step(action)
        Q

In [5]:
def optimal_policy(state, Q):
    action = np.argmax(Q[state])
    return action

In [6]:
bins = np.linspace(-0.000001, 100., 2)
bins

array([-1.e-06,  1.e+02])

In [7]:
def get_state(obs):
    d = np.digitize(obs, bins)
    state = tuple(d)
    return state

In [8]:
state = get_state(np.array([-1.4, -2., 0.23, 1.2]))
state

(0, 0, 1, 1)

In [3]:
Q = np.random.random((2,2,2,2,2))
Q

KeyboardInterrupt: 

In [ ]:
obs = env.reset()
print(obs)
done = False
while not done:
    state = get_state(obs)
    action = epsilon_greedy_policy(state, Q, 0.5)
    obs, reward, done, info = env.step(action)
    print('->', state, action, reward, obs, done, info)
env.close()

[-0.03388138  0.02983903 -0.03020782  0.0430648 ]
explore
-> (0, 1, 0, 1) 0 1.0 [-0.0332846  -0.16483702 -0.02934653  0.3260659 ] False {}
explore
-> (0, 0, 0, 1) 0 1.0 [-0.03658134 -0.3595291  -0.02282521  0.60935163] False {}
exploit
-> (0, 0, 0, 1) 1 1.0 [-0.04377192 -0.16409566 -0.01063818  0.30956778] False {}
exploit
-> (0, 0, 0, 1) 1 1.0 [-0.04705384  0.03117625 -0.00444682  0.01354896] False {}
exploit
-> (0, 1, 0, 1) 1 1.0 [-0.04643031  0.22636169 -0.00417584 -0.28053367] False {}
explore
-> (0, 1, 0, 0) 0 1.0 [-0.04190307  0.03129955 -0.00978651  0.01082929] False {}
exploit
-> (0, 1, 0, 1) 1 1.0 [-0.04127708  0.22656047 -0.00956993 -0.28492525] False {}
explore
-> (0, 1, 0, 0) 0 1.0 [-0.03674588  0.03157631 -0.01526843  0.00472407] False {}
exploit
-> (0, 1, 0, 1) 1 1.0 [-0.03611435  0.22691387 -0.01517395 -0.29273686] False {}
exploit
-> (0, 1, 0, 0) 1 1.0 [-0.03157607  0.42224884 -0.02102869 -0.59016657] False {}
exploit
-> (0, 1, 0, 0) 1 1.0 [-0.02313109  0.6176588  -0.03

: 